In [8]:
# Python 2/3 compatibility
from __future__ import print_function

import numpy as np
import cv2
import video
from common import anorm2, draw_str

In [9]:
lk_params = dict( winSize  = (5, 5),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.2,
                       minDistance = 2,
                       blockSize = 2 )


In [14]:
rows = 250
cols = 250

rec_sizex = 8
rec_sizey = 50
rec_posx = 1
rec_posy = 100
mov_size = 10

detect_interval = 10

# Create some random colors
color = np.random.randint(0,255,(20,1))

#create a empty img and draw a rectangle 
gray = np.zeros((rows,cols),np.uint8)
cv2.rectangle(gray,(rec_posx,rec_posy),(rec_posx+rec_sizex,rec_posy+rec_sizey),255,-1)
rec_posx+= mov_size

#find corners in the created frame
p0 = cv2.goodFeaturesToTrack(gray, mask = None, **feature_params) 

#create a mask for drawing tracks later
mask = np.zeros_like(gray)

cv2.startWindowThread()
cv2.namedWindow("previmg")
cv2.namedWindow("img")


while (rec_posx < cols-rec_sizex):
    
    #move the rectangle by 'mov_size" pixels to the left
    prevgray = gray
    gray = np.zeros((rows,cols),np.uint8)
    cv2.rectangle(gray,(rec_posx,rec_posy),(rec_posx+rec_sizex,rec_posy+rec_sizey),255,-1)
    rec_posx += mov_size
    
    #calc sparse optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(prevgray, gray, p0, None, **lk_params)
    
    #select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    #draw the movement tracks
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        gray = cv2.circle(gray,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(gray,mask)

    #update the new points to track
    p0 = good_new.reshape(-1,1,2)
    
    
    #prints the img
    cv2.imshow('img',img)
    cv2.imshow('previmg',prevgray)
    ch = cv2.waitKey(0)
    if ch == 27:
        break
cv2.destroyAllWindows()

P0:
[[[   2.  101.]]

 [[   9.  101.]]

 [[   2.  150.]]

 [[   9.  150.]]]
P1:
[[[  12.00003052  101.00010681]]

 [[  19.00014687  101.00049591]]

 [[  11.99997902  150.        ]]

 [[  19.00017357  149.99942017]]]
P0 good:
[[   2.  101.]
 [   9.  101.]
 [   2.  150.]
 [   9.  150.]]
P1 good:
[[  12.00003052  101.00010681]
 [  19.00014687  101.00049591]
 [  11.99997902  150.        ]
 [  19.00017357  149.99942017]]
P0:
[[[  12.00003052  101.00010681]]

 [[  19.00014687  101.00049591]]

 [[  11.99997902  150.        ]]

 [[  19.00017357  149.99942017]]]
P1:
[[[  22.34697342   95.54434967]]

 [[  28.8233242    95.48403168]]

 [[  20.45439148  147.15583801]]

 [[  25.53540039  146.38352966]]]
P0 good:
[[  12.00003052  101.00010681]
 [  19.00014687  101.00049591]
 [  11.99997902  150.        ]
 [  19.00017357  149.99942017]]
P1 good:
[[  22.34697342   95.54434967]
 [  28.8233242    95.48403168]
 [  20.45439148  147.15583801]
 [  25.53540039  146.38352966]]
P0:
[[[  22.34697342   95.544349